In [1]:
%matplotlib notebook
import matplotlib
import matplotlib.pyplot as plt
import sys
import numpy as np
import sympy as sym
from sympy.printing import octave_code, ccode
import copy
import scipy

new_paths = ['../Misc/', '../src', '../src/faultdiagnosistoolbox/build/lib.macosx-10.6-x86_64-2.7/', 'build/lib.macosx-10.6-x86_64-2.7/']
[sys.path.append(d) for d in new_paths if not d in sys.path];
from misc import *
import faultdiagnosistoolbox as fdt
import GetMeasurementData as gm

In [3]:
import VEP4Engine
reload(VEP4Engine)
model = VEP4Engine.model;
model.Lint()

Model: VEP4

  Type:Symbolic, dynamic

  Variables and equations
    90 unknown variables
    10 known variables
    11 fault variables
    94 equations, including 14 differential constraints

  Degree of redundancy: 4
  Degree of redundancy of MTES set: 1


  Model validation finished with 0 errors and 0 warnings.


In [4]:
dataDir = '/Users/frisk/Diagnos/Work/EngineDiagnosis/Work/Data/'
dataSets = {'NF' : 'driving_cycle_FTP75_highway_no_fault_dataset1_16-01-20.mat',
            'fyp_im' : 'driving_cycle_FTP75Highway_fault_y_pim_dataset_7_26-01-2016.mat',
            'fyw_af' : 'driving_cycle_FTP75Highway_fault_y_waf_dataset_5_26-01-2016',
            'fyp_ic' : 'driving_cycle_FTP75Highway_fault_y_pic_dataset_8_26-01-2016',
            'fyT_ic' : 'driving_cycle_FTP75Highway_fault_y_Tic_dataset_6_26-01-2016'}
data = {}
sys.stdout.write('Loading data ')
for dd in dataSets:
    sys.stdout.write('.')
    data[dd] = gm.GetMeasurementData(dataDir + dataSets[dd])
sys.stdout.write(' Finished loading ' + str(len(data.keys())) + ' datasets\n')

Loading data ..... Finished loading 5 datasets


In [5]:
print "Searchin for MSO sets..."
msos = model.MSO()
print "Found " + str(len(msos)) + " MSO sets"

Searchin for MSO sets...
Found 4496 MSO sets


In [ ]:
# MSO selected in simple Kulback-Leibler selection strategy
ts = [ 1649, 4011, 4016, 4017, 4066, 4074, 4477]
re = [ 73  , 75  , 75  , 3   , 76  , 3   , 2]

In [ ]:
test = ts[0]
mso = msos[test]
red = mso[re[0]]
m0 = [e for e in mso if e != red]
resName = "ResGen_" + str(test) + "_" + str(red)
Gamma = model.Matching(m0)
model.SeqResGen(Gamma, red, resName, language='Python', batch=False, external_src=['extmodelfuns.py'])
model.SeqResGen(Gamma, red, resName + '_b', language='Python', batch=True, external_src=['extmodelfuns.py'])
model.SeqResGen(Gamma, red, resName + "c", language='C', batch=False, user_functions=VEP4Engine.ext_funs,
               external_headers=['extmodelfuns.h'], external_src=['extmodelfuns.cc'])
model.SeqResGen(Gamma, red, resName + "c_b", language='C', batch=True, user_functions=VEP4Engine.ext_funs,
               external_headers=['extmodelfuns.h'], external_src=['extmodelfuns.cc'])

In [ ]:
import ResGen_1649_1
import ResGen_1649_1_b
import ResGen_1649_1c
import ResGen_1649_1c_b
reload(ResGen_1649_1);
reload(ResGen_1649_1_b);

In [ ]:
ds = data['NF']
N = len(ds['time'])
r = np.zeros(N)
Ts = ds['Ts']
state = ds['state_init'].copy()
t = Timer()

t.tic()
for k,zk in enumerate(ds['z']):
    if np.mod(k,N/20)==0:
        sys.stdout.write('%.0f%% ' % (k/float(N)*100))
    r[k],state = ResGen_1649_1.ResGen_1649_1(zk, 
                                state, 
                                VEP4Engine.diag_par, 
                                Ts)
sys.stdout.write('\n')
print "Residual generation completed in %.1f seconds" % (t.toc())

In [ ]:
t.tic()
state = ds['state_init'].copy()
r_b = ResGen_1649_1_b.ResGen_1649_1_b(ds['z'],state,VEP4Engine.diag_par,Ts)
print "Residual generation completed in %.1f seconds" % (t.toc())

In [ ]:
rc = np.zeros(N)
state = ds['state_init'].copy()
t.tic()
for k,zk in enumerate(ds['z']):
    if np.mod(k,N/20)==0:
        sys.stdout.write('%.0f%% ' % (k/float(N)*100))
    rc[k],state = ResGen_1649_1c.ResGen_1649_1c(zk, 
                                state, 
                                VEP4Engine.diag_par, 
                                Ts)
sys.stdout.write('\n')
print "Residual generation completed in %.1f seconds" % (t.toc())

In [ ]:
t.tic()
state = ds['state_init'].copy()
rc_b = ResGen_1649_1c_b.ResGen_1649_1c_b(ds['z'],state,VEP4Engine.diag_par,Ts)
print "Residual generation completed in %.3f seconds" % (t.toc())

In [ ]:
plt.figure(10)
plt.plot(r[::1000])
plt.plot(r_b[::1000])
plt.plot(rc[::1000])
plt.plot(rc_b[::1000])


In [ ]:
print np.linalg.norm(r-r_b)
print np.linalg.norm(rc-rc_b)
print np.linalg.norm(r-rc)

In [ ]:
plt.figure(20)
plt.plot(r-rc)